### **FUENTES**:

PetFinder Kaggle:

https://www.kaggle.com/competitions/petfinder-adoption-prediction/data

First Tutorial:

https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5

Second Deep Tutorial:

https://rumn.medium.com/part-1-ultimate-guide-to-fine-tuning-in-pytorch-pre-trained-model-and-its-configuration-8990194b71e

Logo Recognition API:

https://heartbeat.comet.ml/logo-recognition-ios-application-using-machine-learning-and-flask-api-aec4eff3be11

Hybrid (multimodal) neural network architecture : Combination of tabular, textual and image inputs:

https://medium.com/@dave.cote.msc/hybrid-multimodal-neural-network-architecture-combination-of-tabular-textual-and-image-inputs-7460a4f82a2e



### **INDICACIONES PREVIAS**:

+ **Git**:
    + Clonamos el repo: root de todos los repos y ponemos git clone "url_repo"
    + Hacemos el checkout de la rama main: git checkout -b new-branch

+ **Poetry**:
    + Instalamos poetry: https://python-poetry.org/docs/
    + Realizamos un Update del pyproject: poetry update
    + Activamos el entorno que creo poetry: poetry shell
    + Intentamos correr una celda, si nos pide seleccionar el environment y no lo vemos en la lista, cerrar y volver abrir VSC

+ **Torch y CUDA**:
    + Verificar que versión pide torch:
        + Versión de torch instalada: poetry show (en mi caso la 1.13.1)
        + Buscar la versión correspondiente en la documentación: https://pytorch.org/get-started/previous-versions/  (en mi caso el 11.7)
    + Instalar CUDA para Torch (buscar la versión correspondiente de CUDA): https://developer.nvidia.com/cuda-11-7-0-download-archive
    + Verificar que CUDA esté funcional: correr en una celda torch.cuda.is_available()

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import os
import shutil
import time
import copy
import datetime
from tqdm import tqdm
#import matplotlib.pyplot as plt
#import seaborn as sns
#import cv2
#from PIL import Image
#from pathlib import Path

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F

from joblib import load, dump

from utils import plot_confusion_matrix
# Verificamos que CUDA está funcional
torch.cuda.is_available()

/home/aa/miniconda3/envs/ldi2_cuda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

**Seteo el Modelo**

Teoría de Resnet: https://towardsdatascience.com/introduction-to-resnets-c0a830a288a4

In [2]:
# Importo modelo ResNet entrenado en Imagenet
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# Modificar la última capa para adaptarse a tu problema específico
num_ftrs = resnet50.fc.in_features
resnet50.fc = torch.nn.Linear(num_ftrs, 5) # Clasificación 5 clases
# Configuro para usar cuda si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)
# Instancio del criterio de pérdida CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
# Instancio Stochastic Gradient Descent (SGD): Defino el parámetro del Learning Rate (define "el paso" en que avanzan los pesos en cada iteración) y el Momentum (pone innercia a la dirección del gradiente descendiente para que no cambie de dirección en minimos locales)
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9) # Parámetros default del SGD


In [3]:
torch.__version__

'2.2.2'

**Seteo parámetros, directorios y funciones**

In [4]:
# Paths
BASE_DIR = '../'
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")
PATH_TO_IMAGES_DIR = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train_images")
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")

MODEL_NAME = '04 ResNet'

MODEL_VERSION = '1.0.0'

# Parametros y variables
CREATE_PYTORCH_DIRECTORIES = 1
SEED = 42
BATCH_SIZE = 50
TEST_SIZE = 0.2
IMAGE_SIZE = 299
CPU_CORES = os.cpu_count()

# Armo el nuevo directorio de train
new_train_directory = os.path.join(BASE_DIR, 'work/train_images_classes')
os.makedirs(new_train_directory, exist_ok=True) # si ya existe el nombre, lo deja como está

# Armo el nuevo directorio de validación
new_val_directory = os.path.join(BASE_DIR, 'work/val_images_classes')
os.makedirs(new_val_directory, exist_ok=True)

# Definir las clases ordenadas
class_names = ['0', '1', '2', '3', '4']

# Mapear las etiquetas de las clases a números enteros consecutivos
class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

# Creo las carpetas de clases dentro de los directorios
for clase in class_names: # Una para cada clase
   os.makedirs(os.path.join(new_train_directory, str(clase)), exist_ok=True)
   os.makedirs(os.path.join(new_val_directory, str(clase)), exist_ok=True)




# Funciones para la carga y el preproceso
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    # Calcula el factor de escala necesario para redimensionar la imagen de manera que el lado más largo tenga el tamaño deseado 
    ratio = float(IMAGE_SIZE)/max(old_size)
    # Calcula las nuevas dimensiones de la imagen 
    new_size = tuple([int(x*ratio) for x in old_size])
    # Redimensiona la imagen con el nuevo tamaño
    im = cv2.resize(im, (new_size[1], new_size[0]))
    # Calcula las diferencias de tamaño y agrega pixeles (color negro) en los extremos para que quede centrada y cuadrada 
    delta_w = IMAGE_SIZE - new_size[1]
    delta_h = IMAGE_SIZE - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_image = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_image


def load_image(pet_id):
    path_to_image = os.path.join(PATH_TO_IMAGES_DIR, f'{pet_id}-1.jpg') # Irá a la primera imagen de la mascota
    image = cv2.imread(path_to_image)
    # Convierte la imagen de BGR a RGB porque estos modelos esperan ese orden de canales
    image = cv2.convertScaleAbs(image)
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    new_image = resize_to_square(image)
    return new_image


In [5]:

def visualize_pet(pet_id):
    path_to_image = os.path.join(PATH_TO_IMAGES_DIR, f'{pet_id}-1.jpg') # Irá a la primera imagen de la mascota
    # Cargar la imagen
    image_to_show = cv2.imread(path_to_image)
    # Convertir a formato RGB
    image_to_show = cv2.cvtColor(image_to_show, cv2.COLOR_BGR2RGB)
    # Visualizar la imagen
    plt.imshow(image_to_show)
    plt.axis('off')  # No mostrar los ejes
    plt.show()

def visualize_image(image):
    # Convierte la imagen a un formato de enteros (CV_8U)
    image = cv2.convertScaleAbs(image)
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Visualizar la imagen
    plt.imshow(image.astype(np.uint8))
    plt.axis('off')  # No mostrar los ejes
    plt.show()


**Cargo y Proceso Data**

Nota: Pytorch necesita que estén las imágenes en los distintos directorios según su clase y su participación en el training

In [6]:
# Cargo
train_df = pd.read_csv(PATH_TO_TRAIN)

# Split para validación
train_data, val_data = train_test_split(train_df,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               stratify = train_df.AdoptionSpeed)




if CREATE_PYTORCH_DIRECTORIES == 1: # Poner en 0 si ya tengo las carpetas train_images_classes y val_images_classes con las imágenes copiadas
    # Función para copiar las imágenes a los directorios correspondientes
    def copy_imag(data, directorio_destino):
        for index, row in data.iterrows():
            petID = row['PetID']
            adoption_speed = row['AdoptionSpeed']
            
            # Nombre del archivo de imagen
            nombre_archivo = f"{petID}-1.jpg"
            
            # Ruta completa de la imagen de origen
            ruta_origen = os.path.join(PATH_TO_IMAGES_DIR, nombre_archivo)
            
            # Ruta completa del directorio de destino
            ruta_destino = os.path.join(directorio_destino, str(adoption_speed), nombre_archivo)
            
            # Verificar si el archivo de origen existe
            if os.path.exists(ruta_origen):
                # Copiar el archivo de origen al directorio de destino
                shutil.copy2(ruta_origen, ruta_destino)
        print("Completada la copia a: ",str(directorio_destino))

    # Copiar las imágenes al directorio de train
    copy_imag(train_data, new_train_directory)

    # Copiar las imágenes al directorio de val
    copy_imag(val_data, new_val_directory)

    print("Proceso completado.")

Completada la copia a:  ../work/train_images_classes
Completada la copia a:  ../work/val_images_classes
Proceso completado.


In [7]:
# Genero los DataLoaders
def create_dataloaders(train_directory, val_directory, batch_size, num_workers):
    # Transformaciones de imagen para el conjunto de entrenamiento
    train_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Transformaciones de imagen para el conjunto de validación (sin data augment)
    val_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Crear conjuntos de datos para el conjunto de entrenamiento y validación
    conjunto_entrenamiento = datasets.ImageFolder(train_directory, transform=train_transforms)
    conjunto_validacion = datasets.ImageFolder(val_directory, transform=val_transforms)

    # Asignar las clases ordenadas al conjunto de datos
    conjunto_entrenamiento.class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}
    conjunto_validacion.class_to_idx = {class_name: i for i, class_name in enumerate(class_names)}

    # Crear dataloaders para el conjunto de entrenamiento y validación
    train_dataloader = torch.utils.data.DataLoader(conjunto_entrenamiento, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_dataloader = torch.utils.data.DataLoader(conjunto_validacion, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_dataloader, val_dataloader

# Aplico las funcion de los DataLoaders
train_dataloader, val_dataloader = create_dataloaders(new_train_directory , new_val_directory , BATCH_SIZE, CPU_CORES)

In [8]:
#Genero una lista de PetIDs con imagen en el orden en que aparecen en el data loader
test_sample_ids = [i[0].split('/')[-1].split('-')[0] for i in val_dataloader.dataset.samples]

**Entreno**

In [9]:
def train_val(model, criterion, optimizer, dataloaders, datasets, device, num_epochs=20, lr=0.001, momentum = 0.9 ,trial=None):
    
    # Instancio Stochastic Gradient Descent (SGD): Defino el parámetro del Learning Rate (define "el paso" en que avanzan los pesos en cada iteración) y el Momentum (pone innercia a la dirección del gradiente descendiente para que no cambie de dirección en minimos locales)
    optimizer = optim.SGD(resnet50.parameters(), lr=lr, momentum=momentum) # Parámetros default del SGD
    
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_kappa =  -999

    train_losses = []
    val_losses = []

    try:
        previous_best = study.best_value
    except:
        previous_best = -999


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        kappa_labels_true = []
        kappa_labels_predicted = []
        output_scores = []

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)



                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    elif phase == 'val':
                        kappa_labels_true.extend(labels.cpu().numpy().tolist())
                        kappa_labels_predicted.extend(preds.cpu().numpy().tolist())
                        outputs_np = outputs.cpu().numpy()
                        output_scores.extend([outputs_np[i,:] for i in range(outputs_np.shape[0])])

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                #END OF BATCH

            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc = running_corrects.double() / len(datasets[phase])
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                kappa_score = np.nan
            else:
                val_losses.append(epoch_loss)
                kappa_score = cohen_kappa_score(kappa_labels_true,
                                  kappa_labels_predicted,
                                  weights = 'quadratic')
                    


            print(f'{phase.title()} Loss: {epoch_loss:.4f} Acc: {epoch_acc*100:.2f}% Kappa: {kappa_score:.3f}')

            # If this is the best Epoch so far -> Deep copy the model
            if phase == 'val' and kappa_score > best_kappa:
                best_acc = epoch_acc
                best_kappa = kappa_score
                best_model_wts = copy.deepcopy(model.state_dict())


                #Best Epoch within a trial and better than previous trials
                if trial is not None and best_kappa > previous_best:

                    #Save test dataset with predictions
                    predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
                    predicted_df = pd.DataFrame({'PetID':test_sample_ids,
                                'pred':output_scores}).merge(val_data, on='PetID')
                    dump(predicted_df, predicted_filename)

                    #Generate and save CM 
                    cm_filename = os.path.join(PATH_TO_TEMP_FILES,f'cm_{trial.study.study_name}_{trial.number}.jpg')
                    plot_confusion_matrix(kappa_labels_true,kappa_labels_predicted).write_image(cm_filename)

            #END OF PHASE

        #END OF EPOCH

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.2f}%'.format(best_acc * 100))

    # Load best model weights
    model.load_state_dict(best_model_wts)

    # Save in optuna trial the best test dataset, cm and model weights
    if trial is not None and best_kappa > previous_best:
        upload_artifact(trial, predicted_filename, artifact_store)   

        upload_artifact(trial, cm_filename, artifact_store)

        file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{trial.number}.pth'
        model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
        torch.save(model, model_path) # Podemos guardar solo los pesos si queremos: best_model.state_dict()
        upload_artifact(trial, model_path, artifact_store)

    return model,best_kappa

best_model,_ = train_val(resnet50, criterion, optimizer,
                       dataloaders={'train': train_dataloader, 
                                    'val': val_dataloader}, 
                       datasets={'train': train_data, 'val': val_data}, 
                       device=device, 
                       num_epochs=4)
# Guardo el modelo
run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'{MODEL_NAME}_{MODEL_VERSION}_{run_id}.pth'
model_path = os.path.join(PATH_TO_TEMP_FILES, file_name)
torch.save(best_model, model_path) # Podemos guardar solo los pesos si queremos: best_model.state_dict()
print(f'Modelo guardado en {model_path}')

Epoch 0/3
----------


100%|██████████| 235/235 [00:54<00:00,  4.35it/s]


Train Loss: 1.4227 Acc: 30.42% Kappa: nan


100%|██████████| 59/59 [00:07<00:00,  8.40it/s]


Val Loss: 1.3882 Acc: 33.74% Kappa: 0.252
Epoch 1/3
----------


...

Val Loss: 1.3653 Acc: 34.31% Kappa: 0.300
Epoch 3/3
----------


100%|██████████| 235/235 [00:53<00:00,  4.41it/s]


Train Loss: 1.3156 Acc: 39.56% Kappa: nan


100%|██████████| 59/59 [00:06<00:00,  8.48it/s]


Val Loss: 1.3614 Acc: 35.85% Kappa: 0.304
Training complete in 4m 2s
Best val Acc: 35.85%
Modelo guardado en ../work/optuna_temp_artifacts/04 ResNet_1.0.0_20240824_085109.pth


In [10]:
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)


def optuna_train(trial):

    epochs = trial.suggest_int('epochs', 5, 5)

    lr = trial.suggest_float('lr', 0.00001, 0.1, log=True)

    momentum = trial.suggest_float('momentum', 0.0, 0.95)

    _,best_score = train_val(resnet50, criterion, optimizer,
                       dataloaders={'train': train_dataloader, 
                                    'val': val_dataloader}, 
                       datasets={'train': train_data, 'val': val_data}, 
                       device=device, 
                       num_epochs=epochs,
                       lr=lr,
                       momentum = momentum,
                       trial=trial)


    return(best_score)

/tmp/ipykernel_703451/3870832202.py:1: ExperimentalWarning: FileSystemArtifactStore is experimental (supported from v3.3.0). The interface can change in the future.
  artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)


In [11]:
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///../work/db.sqlite3",  # Specify the storage URL here.
                            study_name=f'{MODEL_NAME}_{MODEL_VERSION}',
                            load_if_exists = True)
study.optimize(optuna_train, n_trials=30)

[I 2024-08-24 08:51:09,743] Using an existing study with name '04 ResNet_1.0.0' instead of creating a new one.


Epoch 0/4
----------


100%|██████████| 235/235 [00:53<00:00,  4.40it/s]


Train Loss: 1.2905 Acc: 41.32% Kappa: nan


100%|██████████| 59/59 [00:06<00:00,  8.49it/s]


...

100%|██████████| 235/235 [00:52<00:00,  4.45it/s]


Train Loss: 0.0063 Acc: 97.53% Kappa: nan


100%|██████████| 59/59 [00:06<00:00,  8.50it/s]


Val Loss: 3.4895 Acc: 31.88% Kappa: 0.226
Epoch 2/4
----------


  0%|          | 0/235 [00:00<?, ?it/s]